NORMALIZE TWEETS

In [2]:
!pip install emoji

   ---------------------------------------- 0.0/586.9 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/586.9 kB ? eta -:--:--
   ----------------------------------- ---- 524.3/586.9 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 586.9/586.9 kB 1.4 MB/s eta 0:00:00


In [3]:
from emoji import demojize
from nltk.tokenize import TweetTokenizer


tokenizer = TweetTokenizer()


def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())


if __name__ == "__main__":
    print(
        normalizeTweet(
            "SC has first two presumptive cases of coronavirus, DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier"
        )
    )

SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL ... via @USER


In [9]:
import pandas as pd
import numpy as np

def load_data_i(file_path, indicators):
    df = pd.read_csv(file_path)
    r_label_cols = ['R1-1-1', 'R2-1-2', 'R3-2-1', 'R4-2-2', 'R5-3-1', 'R6-3-2',
                    'R7-3-3', 'R8-4-1', 'R9-4-2', 'R10-5-1', 'R11-5-2', 'R12-5-3']
    i_label_cols = ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12']

    r_labels = (np.array(df[r_label_cols])).transpose()
    i_labels = (np.array(df[i_label_cols])).transpose()
    texts = np.array([t.strip() for t in df['tokenized tweet']])

    text_input, target_input, labels, target_idx = [], [], [], []
    for i in range(12):
        related_mask = r_labels[i] == 1
        related_num = np.sum(r_labels[i])

        text_input += list(texts[related_mask])
        target_input += [indicators[i]] * related_num
        labels += list(i_labels[i][related_mask])
        target_idx += [i] * related_num

    return text_input, target_input, labels, target_idx

In [11]:
import torch
from torch import nn
from transformers import AutoModel, BertModel, RobertaModel
import os



class IdeologyNet(nn.Module):
    def __init__(self, ):
        super(IdeologyNet, self).__init__()

        self.plm_encoder = RobertaModel.from_pretrained('roberta-base')

        self.cls_l1 = nn.Linear(768, 128)
        self.cls_l2 = nn.Linear(128, 3)
        self.dropout = nn.Dropout()
        self.act_fun = nn.GELU()
        for param in self.plm_encoder.parameters():
            param.requires_grad = False

    def forward(self, ids_text, mask_text):
        text_embeds = self.plm_encoder(ids_text, mask_text).last_hidden_state

        text_reps = text_embeds[:, 0, :]  # [bs, 768]

        out = self.cls_l1(text_reps)
        out = self.act_fun(out)
        out = self.dropout(out)
        out = self.cls_l2(out)

        return out



In [14]:
import time
import logging
import tqdm
import numpy as np
import torch
from torch import nn
from torch.optim import AdamW
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
from transformers import BertTokenizer, RobertaTokenizer
from sklearn.metrics import f1_score, precision_score, recall_score

def setup_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


setup_seed(42)

# # tokenizer
# if args.plm == 'bertweet':
#     tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')
# elif args.plm == 'bert':
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# elif args.plm == 'roberta':
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


# load indicators
indicators_file = open('C:\Learn\IPD\\backend\\tools\MITweet\data\\random_split\Indicators.txt', encoding='utf-8')
indicators = [' '.join(line.strip('\n').strip().split(' ')[:18]) for line in indicators_file]
indicators = [ind.replace(' ', '</s> ') for ind in indicators]


# load data
train_texts, train_targets, train_labels, train_target_idx = load_data_i('testing\\train.csv', indicators)
val_texts, val_targets, val_labels, val_target_idx = load_data_i('testing\\val.csv', indicators)
test_texts, test_targets, test_labels, test_target_idx = load_data_i('testing\\test.csv', indicators)

train_labels, val_labels, test_labels = torch.LongTensor(train_labels), torch.LongTensor(val_labels), torch.LongTensor(test_labels)
train_target_idx, val_target_idx, test_target_idx = torch.LongTensor(train_target_idx), torch.LongTensor(val_target_idx), torch.LongTensor(test_target_idx)
train_size = len(train_texts)

# loss function
criterion = nn.CrossEntropyLoss().cuda()

# model
net = IdeologyNet().cuda()
multi_gpu = False

# optimizer
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight", "layer_norm.bias", "layer_norm.weight"]
param_all = list(net.module.named_parameters() if multi_gpu else net.named_parameters())
param_plm_encoder = [(n, p) for n, p in param_all if 'encoder' in n]
param_other = [(n, p) for n, p in param_all if not ('encoder' in n)]
del param_all
optimizer_grouped_params = [
        {"params": [p for n, p in param_plm_encoder if not any(nd in n for nd in no_decay)],
         "lr": 2e-5,
         "weight_decay": 0.01},
        {"params": [p for n, p in param_plm_encoder if any(nd in n for nd in no_decay)],
         "lr": 2e-5,
         "weight_decay": 0.0},
        {"params": [p for n, p in param_other if not any(nd in n for nd in no_decay)],
         "lr": 1e-3,
         "weight_decay": 5e-4},
        {"params": [p for n, p in param_other if any(nd in n for nd in no_decay)],
         "lr": 1e-3,
         "weight_decay": 0.0}
]
del param_plm_encoder, param_other
optimizer = AdamW(optimizer_grouped_params, betas=(0.9, 0.999), eps=1e-8)

@torch.no_grad()
def evaluate(texts, targets, labels, target_idx, task='Val'):
    net.eval()
    data_size = len(texts)

    loss_epoch = 0
    pred_epoch = None

    all_indices = torch.arange(data_size).split(32)
    for batch_indices in all_indices:
        batch_text = [texts[i] for i in batch_indices]
        batch_target = [targets[i] for i in batch_indices]
        encode_dict = tokenizer(batch_text, text_pair=batch_target, padding=True, truncation='only_first',
                                max_length=128, return_tensors='pt')

        out = net(encode_dict['input_ids'].cuda(), encode_dict['attention_mask'].cuda())

        batch_label = labels[batch_indices].cuda()
        loss = criterion(out, batch_label)
        loss_epoch += loss.item()

        _, pred = torch.max(out, dim=1)
        if pred_epoch is None:
            pred_epoch = pred.cpu()
        else:
            pred_epoch = torch.cat((pred_epoch, pred.cpu()), dim=0)

    loss_epoch /= (data_size // 32 + 1)
    results = compute_performance(pred_epoch, labels.clone(), target_idx.clone())
    print(f"{task}: loss={loss_epoch:.4f}, avg_acc={results['avg_acc']:.4f}, avg_f1={results['avg_f1']:.4f}, "
             f"global_acc={results['global_acc']:.4f}, global_f1={results['global_f1']:.4f}, "
             f"global_p={results['global_p']:.4f}, global_r={results['global_r']:.4f}, "
             f"acc={results['acc']}, f1={results['f1']}, p={results['p']}, r={results['r']}")

    net.train()

    return results


def train():
    best_epoch = 0
    best_val_global_f1 = 0
    report_avg_acc, report_avg_f1 = 0, 0
    report_global_acc, report_global_f1, report_global_p, report_global_r = 0, 0, 0, 0
    report_acc, report_f1, report_p, report_r = None, None, None, None

    for epoch in range(15):
        print(f'\nEpoch: {epoch+1}')

        print(f"lr_plm: {optimizer.state_dict()['param_groups'][0]['lr']}")
        print(f"lr_other: {optimizer.state_dict()['param_groups'][2]['lr']}")

        loss_epoch = 0
        pred_epoch, truth_epoch, target_idx_epoch = None, None, None

        start = time.time()

        net.train()
        all_indices = torch.randperm(train_size).split(32)
        step = 0
        for batch_indices in tqdm.tqdm(all_indices, desc="batch"):
            step += 1
            batch_text = [train_texts[i] for i in batch_indices]
            batch_target = [train_targets[i] for i in batch_indices]
            encode_dict = tokenizer(batch_text, text_pair=batch_target, padding=True, truncation='only_first',
                                    max_length=128, return_tensors='pt')

            out = net(encode_dict['input_ids'].cuda(), encode_dict['attention_mask'].cuda())

            batch_label = train_labels[batch_indices].cuda()
            loss = criterion(out, batch_label)

            # optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            loss_epoch += loss.item()
            _, pred = torch.max(out, dim=1)
            batch_target_idx = train_target_idx[batch_indices]
            if pred_epoch is None:
                pred_epoch, truth_epoch, target_idx_epoch = pred.cpu(), batch_label.cpu(), batch_target_idx.cpu()
            else:
                pred_epoch = torch.cat((pred_epoch, pred.cpu()), dim=0)
                truth_epoch = torch.cat((truth_epoch, batch_label.cpu()), dim=0)
                target_idx_epoch = torch.cat((target_idx_epoch, batch_target_idx.cpu()), dim=0)

            if step % (4000 // 32) == 0:
                num_steps = 4000 // 32
                loss_epoch /= num_steps
                results = compute_performance(pred_epoch, truth_epoch, target_idx_epoch)
                print(f"Checkpoint: loss={loss_epoch:.4f}, "
                         f"avg_acc={results['avg_acc']:.4f}, avg_f1={results['avg_f1']:.4f}, "
                         f"global_acc={results['global_acc']:.4f}, global_f1={results['global_f1']:.4f}, "
                         f"global_p={results['global_p']:.4f}, global_r={results['global_r']:.4f}, "
                         f"acc={results['acc']}, f1={results['f1']}, p={results['p']}, r={results['r']}")
                loss_epoch = 0
                pred_epoch, truth_epoch, target_idx_epoch = None, None, None

        val_results = evaluate(val_texts, val_targets, val_labels, val_target_idx, task='Val')
        test_results = evaluate(test_texts, test_targets, test_labels, test_target_idx, task='Test')
        if val_results['global_f1'] > best_val_global_f1:
            best_epoch = epoch + 1
            best_val_global_f1 = val_results['global_f1']
            report_avg_acc, report_avg_f1 = test_results['avg_acc'], test_results['avg_f1']
            report_global_acc, report_global_f1,  = test_results['global_acc'], test_results['global_f1']
            report_global_p, report_global_r = test_results['global_p'], test_results['global_r']
            report_acc, report_f1, report_p, report_r = test_results['acc'], test_results['f1'], test_results['p'], test_results['r']

        end = time.time()
        print('Training Time: {:.2f}s'.format(end - start))

    print(f"\nReport: best_epoch={best_epoch}\n"
             f"avg_acc={report_avg_acc:.4f}, avg_f1={report_avg_f1:.4f}, "
             f"global_acc={report_global_acc:.4f}, global_f1={report_global_f1:.4f}, "
             f"global_p={report_global_p:.4f}, global_r={report_global_r:.4f}, "
             f"avg_p={np.mean([x for x in report_p if x >= 0]):.4f}, avg_r={np.mean([x for x in report_r if x >= 0]):.4f}, "
             f"acc={report_acc}, f1={report_f1}, p={report_p}, r={report_r}")


def compute_performance(pred, truth, target_idx):
    num_sample = len(pred)
    num_correct_total = (pred == truth).sum()
    global_acc = num_correct_total / num_sample
    global_f1 = f1_score(truth, pred, average='macro')
    global_p = precision_score(truth, pred, average='macro', zero_division=0)
    global_r = recall_score(truth, pred, average='macro', zero_division=0)

    acc, f1, p, r = [], [], [], []
    for i in range(12):
        mask_i = target_idx == i
        if mask_i.sum() == 0:
            acc.append(-1)
            f1.append(-1)
            p.append(-1)
            r.append(-1)
            continue
        pred_i = pred[mask_i]
        truth_i = truth[mask_i]
        label_set_i = list(set(truth_i.numpy()))  # labels that present in current dimension
        assert mask_i.sum() == len(pred_i)
        acc.append(((pred_i == truth_i).sum() / mask_i.sum()).item())
        f1.append(f1_score(truth_i, pred_i, average='macro', labels=label_set_i))
        p.append(precision_score(truth_i, pred_i, average='macro', labels=label_set_i, zero_division=0))
        r.append(recall_score(truth_i, pred_i, average='macro', labels=label_set_i, zero_division=0))

    avg_acc = np.mean([x for x in acc if x >= 0])
    avg_f1 = np.mean([x for x in f1 if x >= 0])

    result_dict = {'avg_acc': avg_acc, 'avg_f1': avg_f1, 'global_acc': global_acc, 'global_f1': global_f1,
                   'global_p': global_p, 'global_r': global_r,
                   'acc': [round(x, 4) for x in acc], 'f1': [round(x, 4) for x in f1], 'p': [round(x, 4) for x in p],
                   'r': [round(x, 4) for x in r]}

    return result_dict

c:\Users\ranah\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   6%|6         | 31.5M/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
train()


Epoch: 1
lr_plm: 2e-05
lr_other: 0.001


batch:  33%|███▎      | 125/375 [01:18<02:46,  1.50it/s]

Checkpoint: loss=0.9935, avg_acc=0.4850, avg_f1=0.2748, global_acc=0.5195, global_f1=0.3532, global_p=0.4127, global_r=0.3905, acc=[0.52, 0.2877, 0.3892, 0.651, 0.6018, 0.3684, 0.5455, 0.369, 0.2606, 0.6798, 0.7997, 0.3474], f1=[0.4792, 0.2111, 0.2688, 0.3144, 0.3177, 0.1944, 0.2353, 0.2631, 0.2163, 0.2901, 0.3048, 0.2023], p=[0.55, 0.293, 0.3415, 0.2828, 0.3582, 0.1458, 0.1875, 0.3078, 0.3974, 0.3389, 0.2971, 0.3105], r=[0.5321, 0.3453, 0.3497, 0.3572, 0.3423, 0.2917, 0.3158, 0.308, 0.3518, 0.3229, 0.316, 0.3198]


batch:  67%|██████▋   | 250/375 [02:45<02:02,  1.02it/s]

Checkpoint: loss=0.8847, avg_acc=0.5492, avg_f1=0.2686, global_acc=0.6340, global_f1=0.4542, global_p=0.4807, global_r=0.4902, acc=[0.4375, 0.2069, 0.3791, 0.7746, 0.5849, 0.5, 0.6176, 0.3908, 0.4846, 0.6732, 0.8586, 0.6827], f1=[0.2029, 0.1358, 0.2355, 0.3747, 0.2796, 0.2222, 0.2545, 0.2894, 0.2831, 0.288, 0.3462, 0.3108], p=[0.1458, 0.1538, 0.2172, 0.3373, 0.2552, 0.1944, 0.2059, 0.3507, 0.3524, 0.3377, 0.3345, 0.4743], r=[0.3333, 0.2764, 0.3327, 0.441, 0.3143, 0.2593, 0.3333, 0.3365, 0.3448, 0.3403, 0.3591, 0.3391]


...

Checkpoint: loss=0.7809, avg_acc=0.5601, avg_f1=0.2830, global_acc=0.6823, global_f1=0.5230, global_p=0.5871, global_r=0.5468, acc=[0.3793, 0.3824, 0.3671, 0.7219, 0.627, 0.375, 0.5306, 0.366, 0.5728, 0.6851, 0.8707, 0.843], f1=[0.2569, 0.2831, 0.3013, 0.2795, 0.2818, 0.1818, 0.2561, 0.3129, 0.278, 0.3048, 0.3287, 0.3307], p=[0.3283, 0.2637, 0.3347, 0.2455, 0.2693, 0.1429, 0.2561, 0.3521, 0.3115, 0.475, 0.6278, 0.3203], r=[0.3074, 0.3444, 0.3442, 0.3244, 0.3289, 0.25, 0.3049, 0.3322, 0.3199, 0.3479, 0.3377, 0.3431]


batch: 100%|██████████| 375/375 [07:34<00:00,  1.21s/it]

Checkpoint: loss=0.7616, avg_acc=0.5682, avg_f1=0.2899, global_acc=0.6871, global_f1=0.5226, global_p=0.5979, global_r=0.5492, acc=[0.44, 0.3, 0.322, 0.7544, 0.6192, 0.4118, 0.5758, 0.3725, 0.5768, 0.7121, 0.8847, 0.8498], f1=[0.3115, 0.2459, 0.2701, 0.347, 0.2756, 0.1944, 0.2962, 0.2858, 0.3348, 0.2888, 0.3129, 0.3159], p=[0.3492, 0.2033, 0.3047, 0.4191, 0.2639, 0.1556, 0.281, 0.3349, 0.4617, 0.2958, 0.2979, 0.3046], r=[0.3968, 0.3212, 0.3054, 0.3678, 0.327, 0.2593, 0.3274, 0.3034, 0.3688, 0.3349, 0.3296, 0.3283]


Val: loss=0.7053, avg_acc=0.5864, avg_f1=0.2754, global_acc=0.7176, global_f1=0.5831, global_p=0.6499, global_r=0.5881, acc=[0.4375, 0.2083, 0.4492, 0.7604, 0.6629, 0.4545, 0.5455, 0.4027, 0.6162, 0.7224, 0.8745, 0.902], f1=[0.2738, 0.1642, 0.3049, 0.288, 0.2658, 0.2083, 0.2353, 0.2932, 0.3651, 0.2796, 0.311, 0.3162], p=[0.4667, 0.2117, 0.2958, 0.2535, 0.221, 0.1515, 0.1818, 0.2712, 0.3743, 0.2408, 0.2915, 0.3007], r=[0.381, 0.2701, 0.3759, 0.3333, 0.3333, 0.3333, 0.3333, 0.3486, 0.375, 0.3333, 0.3333, 0.3333]


KeyboardInterrupt: 